In [5]:
!nvidia-smi

import torch
print("torch version:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
print("gpu name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else None)

Mon Nov 24 09:57:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
from google.colab import drive
drive.mount('/content/drive')

from pathlib import Path
DATA_DIR = Path("/content/drive/MyDrive/OULAD")
print(DATA_DIR)
print(list(DATA_DIR.glob("*.csv")))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/OULAD
[PosixPath('/content/drive/MyDrive/OULAD/assessments.csv'), PosixPath('/content/drive/MyDrive/OULAD/studentAssessment.csv'), PosixPath('/content/drive/MyDrive/OULAD/courses.csv'), PosixPath('/content/drive/MyDrive/OULAD/studentRegistration.csv'), PosixPath('/content/drive/MyDrive/OULAD/studentInfo.csv'), PosixPath('/content/drive/MyDrive/OULAD/vle.csv'), PosixPath('/content/drive/MyDrive/OULAD/studentVle.csv')]


In [7]:
from __future__ import annotations
from pathlib import Path
from typing import List, Optional, Dict

import numpy as np
import pandas as pd

SNAPSHOT_OFFSETS_DAYS: List[int] = [7, 3, 1, 0]
WARN_HOURS: int = 48

In [8]:
def load_oulad(raw_dir: Path,
               modules: Optional[List[str]] = None,
               presentations: Optional[List[str]] = None
               ) -> Dict[str, pd.DataFrame]:

    raw_dir = Path(raw_dir)

    assessments = pd.read_csv(raw_dir / "assessments.csv")
    student_assessment = pd.read_csv(raw_dir / "studentAssessment.csv")
    student_reg = pd.read_csv(raw_dir / "studentRegistration.csv")
    student_info = pd.read_csv(raw_dir / "studentInfo.csv")

    if modules is not None:
        assessments = assessments[assessments["code_module"].isin(modules)]
        student_reg = student_reg[student_reg["code_module"].isin(modules)]
        student_info = student_info[student_info["code_module"].isin(modules)]

    if presentations is not None:
        assessments = assessments[assessments["code_presentation"].isin(presentations)]
        student_reg = student_reg[student_reg["code_presentation"].isin(presentations)]
        student_info = student_info[student_info["code_presentation"].isin(presentations)]

    return {
        "assessments": assessments,
        "student_assessment": student_assessment,
        "student_reg": student_reg,
        "student_info": student_info,
    }


def prepare_assessments(assessments: pd.DataFrame) -> pd.DataFrame:
    df = assessments.copy()
    df["course_id"] = df["code_module"] + "_" + df["code_presentation"]

    df = df.rename(columns={
        "id_assessment": "task_id",
        "assessment_type": "task_type",
        "date": "due_day",
    })

    df["due_day"] = pd.to_numeric(df["due_day"], errors="coerce")
    df = df[np.isfinite(df["due_day"])].copy()

    df["weight"] = df["weight"].fillna(0.0)
    df["weight_frac"] = df["weight"] / 100.0
    df["due_day"] = df["due_day"].astype(int)

    return df[[
        "course_id", "code_module", "code_presentation",
        "task_id", "task_type", "due_day", "weight", "weight_frac"
    ]]


def prepare_registrations(student_reg: pd.DataFrame) -> pd.DataFrame:
    df = student_reg.copy()
    df["course_id"] = df["code_module"] + "_" + df["code_presentation"]
    df = df.rename(columns={"id_student": "student_id"})
    return df[["course_id", "student_id", "date_registration", "date_unregistration"]]


def prepare_student_assessment(student_assessment: pd.DataFrame,
                               assessments_pre: pd.DataFrame) -> pd.DataFrame:

    sa = student_assessment.copy().rename(columns={
        "id_student": "student_id",
        "id_assessment": "task_id",
        "date_submitted": "submission_day"
    })

    sa["submission_day"] = pd.to_numeric(sa["submission_day"], errors="coerce")

    sa = sa.merge(
        assessments_pre[["course_id", "task_id", "due_day", "weight_frac"]],
        on="task_id",
        how="left"
    )

    sa["submitted_final"] = sa["submission_day"].notna()
    sa["late_final"] = np.where(
        sa["submitted_final"],
        sa["submission_day"] > sa["due_day"],
        True,
    )

    sa["buffer_days"] = np.where(
        sa["submitted_final"],
        sa["due_day"] - sa["submission_day"],
        np.nan
    )

    return sa[[
        "course_id", "student_id", "task_id",
        "due_day", "submission_day",
        "submitted_final", "late_final", "buffer_days",
        "weight_frac", "score", "is_banked"
    ]]


def prepare_student_info(student_info: pd.DataFrame) -> pd.DataFrame:
    si = student_info.copy().rename(columns={"id_student": "student_id"})
    si["course_id"] = si["code_module"] + "_" + si["code_presentation"]

    return si[[
        "course_id", "student_id",
        "gender", "age_band", "highest_education",
        "studied_credits"
    ]]


def build_assessment_snapshots(assessments_pre: pd.DataFrame,
                               snapshot_offsets: List[int]) -> pd.DataFrame:
    offsets_df = pd.DataFrame({"offset_days": snapshot_offsets})
    a = assessments_pre[["course_id", "task_id", "due_day"]].copy()
    a["key"] = 1
    offsets_df["key"] = 1
    cross = a.merge(offsets_df, on="key").drop(columns="key")

    cross["snapshot_day"] = cross["due_day"] - cross["offset_days"]
    cross = cross[cross["snapshot_day"] >= 0].copy()

    cross["time_to_deadline_days"] = cross["offset_days"]
    cross["time_to_deadline_hours"] = cross["time_to_deadline_days"] * 24

    return cross[[
        "course_id", "task_id", "snapshot_day",
        "time_to_deadline_days", "time_to_deadline_hours",
        "due_day"
    ]]


def attach_students_to_snapshots(snapshots_task: pd.DataFrame,
                                 registrations: pd.DataFrame) -> pd.DataFrame:
    students = registrations[["course_id", "student_id"]].drop_duplicates()
    snapshots = snapshots_task.merge(students, on="course_id", how="inner")
    snapshots["snapshot_id"] = (
        snapshots["student_id"].astype(str) + "_" +
        snapshots["task_id"].astype(str) + "_" +
        snapshots["snapshot_day"].astype(str)
    )
    return snapshots


def add_labels_and_submission_status(snapshots: pd.DataFrame,
                                     sa_pre: pd.DataFrame) -> pd.DataFrame:
    df = snapshots.merge(
        sa_pre[["course_id", "student_id", "task_id",
                "due_day", "submission_day",
                "submitted_final", "late_final"]],
        on=["course_id", "student_id", "task_id"],
        how="left"
    )

    df["submitted_final"] = df["submitted_final"].fillna(False)
    df["late_final"] = df["late_final"].fillna(True)

    df["submitted_by_snapshot"] = np.where(
        df["submitted_final"] & df["submission_day"].notna(),
        df["submission_day"] <= df["snapshot_day"],
        False
    )

    df["label_late_final"] = df["late_final"].astype(int)

    warn_days = WARN_HOURS / 24.0
    df["label_warn_H48"] = np.where(
        (df["time_to_deadline_days"] <= warn_days) &
        (~df["submitted_by_snapshot"]) &
        (df["late_final"]),
        1,
        0
    )
    return df


def build_past_assessment_history(sa_pre: pd.DataFrame) -> pd.DataFrame:
    daily = (
        sa_pre.groupby(["course_id", "student_id", "due_day"], as_index=False)
              .agg(
                  tasks_on_day=("task_id", "count"),
                  late_on_day=("late_final", "sum"),
                  submitted_on_day=("submitted_final", "sum"),
                  buffer_sum_on_day=("buffer_days", lambda x: np.nansum(x))
              )
              .rename(columns={"due_day": "day"})
              .sort_values(["course_id", "student_id", "day"])
    )

    daily["day"] = daily["day"].astype("int64")
    daily["cum_tasks"] = daily.groupby(["course_id", "student_id"])["tasks_on_day"].cumsum()
    daily["cum_late"] = daily.groupby(["course_id", "student_id"])["late_on_day"].cumsum()
    daily["cum_submitted"] = daily.groupby(["course_id", "student_id"])["submitted_on_day"].cumsum()
    daily["cum_buffer_sum"] = daily.groupby(["course_id", "student_id"])["buffer_sum_on_day"].cumsum()
    return daily


def add_past_history_features(snapshots: pd.DataFrame,
                              history_daily: pd.DataFrame) -> pd.DataFrame:
    df = snapshots.copy()
    df["history_day"] = (df["snapshot_day"] - 1).astype("int64")

    hist = history_daily.copy()
    hist["day"] = hist["day"].astype("int64")

    merged = pd.merge_asof(
        df.sort_values("history_day"),
        hist.sort_values("day"),
        left_on="history_day",
        right_on="day",
        by=["course_id", "student_id"],
        direction="backward"
    )

    for col in ["cum_tasks", "cum_late", "cum_submitted", "cum_buffer_sum"]:
        merged[col] = merged[col].fillna(0.0)

    merged["past_num_tasks"] = merged["cum_tasks"]
    merged["past_late_ratio"] = np.where(
        merged["cum_tasks"] > 0,
        merged["cum_late"] / merged["cum_tasks"],
        0.0
    )
    merged["past_avg_buffer_days"] = np.where(
        merged["cum_submitted"] > 0,
        merged["cum_buffer_sum"] / merged["cum_submitted"],
        0.0
    )
    return merged


def add_workload_features(snapshots: pd.DataFrame,
                          assessments_pre: pd.DataFrame,
                          horizons_days: List[int] = [3, 7]) -> pd.DataFrame:
    df = snapshots.copy()
    unique_cd = df[["course_id", "snapshot_day"]].drop_duplicates()

    rows = []
    for course_id, grp in unique_cd.groupby("course_id"):
        due_days = assessments_pre.loc[
            assessments_pre["course_id"] == course_id, "due_day"
        ].values

        for snapshot_day in grp["snapshot_day"]:
            row = {"course_id": course_id, "snapshot_day": snapshot_day}
            for h in horizons_days:
                mask = (due_days > snapshot_day) & (due_days <= snapshot_day + h)
                row[f"num_tasks_due_next_{h}d"] = int(mask.sum())
            rows.append(row)

    workload = pd.DataFrame(rows)
    df = df.merge(workload, on=["course_id", "snapshot_day"], how="left")
    for h in horizons_days:
        df[f"num_tasks_due_next_{h}d"] = df[f"num_tasks_due_next_{h}d"].fillna(0).astype(int)
    return df


def add_student_static_features(snapshots: pd.DataFrame,
                                student_info_pre: pd.DataFrame) -> pd.DataFrame:
    return snapshots.merge(student_info_pre, on=["course_id", "student_id"], how="left")


def build_snapshot_dataset(raw_dir: str | Path) -> pd.DataFrame:
    data = load_oulad(raw_dir)
    assessments_pre = prepare_assessments(data["assessments"])
    reg_pre = prepare_registrations(data["student_reg"])
    sa_pre = prepare_student_assessment(data["student_assessment"], assessments_pre)
    si_pre = prepare_student_info(data["student_info"])

    snapshots_task = build_assessment_snapshots(assessments_pre, SNAPSHOT_OFFSETS_DAYS)
    snapshots = attach_students_to_snapshots(snapshots_task, reg_pre)
    snapshots = add_labels_and_submission_status(snapshots, sa_pre)

    history_daily = build_past_assessment_history(sa_pre)
    snapshots = add_past_history_features(snapshots, history_daily)

    snapshots = add_workload_features(snapshots, assessments_pre, horizons_days=[3, 7])
    snapshots = add_student_static_features(snapshots, si_pre)

    return snapshots


In [9]:
df = build_snapshot_dataset(DATA_DIR)
print(df.shape)
df.head()

/tmp/ipython-input-593068031.py:153: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["submitted_final"] = df["submitted_final"].fillna(False)
/tmp/ipython-input-593068031.py:154: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["late_final"] = df["late_final"].fillna(True)


(1218388, 34)


,course_id,task_id,snapshot_day,time_to_deadline_days,time_to_deadline_hours,due_day_x,student_id,snapshot_id,due_day_y,submission_day,...,cum_buffer_sum,past_num_tasks,past_late_ratio,past_avg_buffer_days,num_tasks_due_next_3d,num_tasks_due_next_7d,gender,age_band,highest_education,studied_credits
0,BBB_2014B,15008,5,7,168,12,391153,391153_15008_5,12.0,12.0,...,0.0,0.0,0.0,0.0,0,1,F,0-35,A Level or Equivalent,120
1,BBB_2014B,15008,5,7,168,12,391556,391556_15008_5,NaN,NaN,...,0.0,0.0,0.0,0.0,0,1,F,35-55,No Formal quals,60
2,BBB_2014B,15008,5,7,168,12,392604,392604_15008_5,12.0,2.0,...,0.0,0.0,0.0,0.0,0,1,F,35-55,Lower Than A Level,90
3,BBB_2014B,15008,5,7,168,12,392733,392733_15008_5,12.0,18.0,...,0.0,0.0,0.0,0.0,0,1,F,35-55,HE Qualification,120
4,BBB_2014B,15008,5,7,168,12,393131,393131_15008_5,12.0,19.0,...,0.0,0.0,0.0,0.0,0,1,F,0-35,Lower Than A Level,60


In [10]:
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler

groups = df["student_id"].values

label_cols = ["label_late_final", "label_warn_H48", "submitted_by_snapshot"]

drop_cols = [
    "snapshot_id", "course_id", "student_id", "task_id",
    "submitted_final", "late_final", "submission_day",
] + label_cols

X_df = df.drop(columns=drop_cols, errors="ignore")

# 범주형 one-hot
X_df = pd.get_dummies(X_df, drop_first=True)

# 결측 처리
X_df = X_df.fillna(0.0)

# 수치 스케일링
scaler = StandardScaler()
X = scaler.fit_transform(X_df.values).astype(np.float32)

Y = df[label_cols].astype(np.float32).values

print("X shape:", X.shape, "Y shape:", Y.shape)


X shape: (1218388, 28) Y shape: (1218388, 3)


In [11]:
import torch
from torch.utils.data import Dataset, DataLoader

class TabularDataset(Dataset):
    def __init__(self, X, Y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.Y = torch.tensor(Y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

In [12]:
import torch.nn as nn

class MultiTaskMLP(nn.Module):
    def __init__(self, input_dim, hidden_dims=[256, 128, 64], dropout=0.2):
        super().__init__()
        layers = []
        prev = input_dim
        for h in hidden_dims:
            layers += [
                nn.Linear(prev, h),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.BatchNorm1d(h),
            ]
            prev = h
        self.backbone = nn.Sequential(*layers)

        # 3개 task head
        self.head_late = nn.Linear(prev, 1)
        self.head_warn = nn.Linear(prev, 1)
        self.head_subm = nn.Linear(prev, 1)

    def forward(self, x):
        h = self.backbone(x)
        late = self.head_late(h)
        warn = self.head_warn(h)
        subm = self.head_subm(h)
        return torch.cat([late, warn, subm], dim=1)

In [13]:
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

def train_one_fold(train_idx, val_idx, fold):
    X_train, Y_train = X[train_idx], Y[train_idx]
    X_val, Y_val     = X[val_idx], Y[val_idx]

    train_ds = TabularDataset(X_train, Y_train)
    val_ds   = TabularDataset(X_val, Y_val)

    train_loader = DataLoader(train_ds, batch_size=1024, shuffle=True, num_workers=2)
    val_loader   = DataLoader(val_ds, batch_size=2048, shuffle=False, num_workers=2)

    model = MultiTaskMLP(input_dim=X.shape[1]).to(device)
    opt = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
    crit = nn.BCEWithLogitsLoss()

    best_val = 1e9
    patience = 3
    bad = 0

    for epoch in range(1, 30+1):
        model.train()
        tr_loss = 0.0
        for xb, yb in train_loader:
            xb = xb.to(device)
            yb = yb.to(device)

            logits = model(xb)
            loss = crit(logits, yb)

            opt.zero_grad()
            loss.backward()
            opt.step()

            tr_loss += loss.item() * len(xb)

        tr_loss /= len(train_ds)

        model.eval()
        va_loss = 0.0
        all_logits, all_y = [], []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb = xb.to(device)
                yb = yb.to(device)
                logits = model(xb)

                loss = crit(logits, yb)
                va_loss += loss.item() * len(xb)

                all_logits.append(torch.sigmoid(logits).cpu().numpy())
                all_y.append(yb.cpu().numpy())

        va_loss /= len(val_ds)
        probs = np.vstack(all_logits)
        ys    = np.vstack(all_y)

        print(f"fold {fold} epoch {epoch} train_loss={tr_loss:.4f} val_loss={va_loss:.4f}")

        # early stopping
        if va_loss < best_val:
            best_val = va_loss
            bad = 0
            best_probs = probs.copy()
            best_ys = ys.copy()
        else:
            bad += 1
            if bad >= patience:
                print("early stop")
                break

    # task별 metric 출력
    out = {}
    for i, name in enumerate(["late", "warn48", "submitted"]):
        p = best_probs[:, i]
        y_true = best_ys[:, i]
        y_pred = (p > 0.5).astype(int)

        auc = roc_auc_score(y_true, p)
        f1  = f1_score(y_true, y_pred)
        acc = accuracy_score(y_true, y_pred)
        out[name] = (auc, f1, acc)
        print(f"[fold {fold}] {name}: AUC={auc:.4f}, F1={f1:.4f}, ACC={acc:.4f}")

    return out


gkf = GroupKFold(n_splits=5)
fold_results = []

for fold, (tr_idx, va_idx) in enumerate(gkf.split(X, Y, groups=groups), 1):
    res = train_one_fold(tr_idx, va_idx, fold)
    fold_results.append(res)

fold_results

device: cuda
fold 1 epoch 1 train_loss=0.2273 val_loss=0.1882
fold 1 epoch 2 train_loss=0.1871 val_loss=0.1820
fold 1 epoch 3 train_loss=0.1815 val_loss=0.1782
fold 1 epoch 4 train_loss=0.1787 val_loss=0.1769
fold 1 epoch 5 train_loss=0.1772 val_loss=0.1755
fold 1 epoch 6 train_loss=0.1759 val_loss=0.1762
fold 1 epoch 7 train_loss=0.1747 val_loss=0.1754
fold 1 epoch 8 train_loss=0.1741 val_loss=0.1742
fold 1 epoch 9 train_loss=0.1735 val_loss=0.1738
fold 1 epoch 10 train_loss=0.1728 val_loss=0.1734
fold 1 epoch 11 train_loss=0.1724 val_loss=0.1725
fold 1 epoch 12 train_loss=0.1721 val_loss=0.1731
fold 1 epoch 13 train_loss=0.1714 val_loss=0.1727
fold 1 epoch 14 train_loss=0.1712 val_loss=0.1724
fold 1 epoch 15 train_loss=0.1709 val_loss=0.1727
fold 1 epoch 16 train_loss=0.1707 val_loss=0.1724
fold 1 epoch 17 train_loss=0.1703 val_loss=0.1711
fold 1 epoch 18 train_loss=0.1699 val_loss=0.1724
fold 1 epoch 19 train_loss=0.1699 val_loss=0.1718
fold 1 epoch 20 train_loss=0.1696 val_loss=0.1

[{'late': (np.float64(0.964164622144893),
   0.9202182302801455,
   0.9056631648063034),
  'warn48': (np.float64(0.9901875387838347),
   0.9230587745472209,
   0.9544977019041365),
  'submitted': (np.float64(0.9684992081177602),
   0.8341036578949499,
   0.9158937951411688)},
 {'late': (np.float64(0.9662098364695384),
   0.9216208037232988,
   0.9075303676953381),
  'warn48': (np.float64(0.9907154112973802),
   0.9238621034919134,
   0.955064018384767),
  'submitted': (np.float64(0.9703546432838905),
   0.8413752608568005,
   0.9179620814182534)},
 {'late': (np.float64(0.9666495304347587),
   0.9203805778885651,
   0.906933797337448),
  'warn48': (np.float64(0.9906701619366358),
   0.9222927971508293,
   0.9545133702129056),
  'submitted': (np.float64(0.9701567833662),
   0.8405587974121528,
   0.9174723813588536)},
 {'late': (np.float64(0.9664527828482126),
   0.9213146921756193,
   0.9078448431523827),
  'warn48': (np.float64(0.9906990132925114),
   0.9236199627414452,
   0.955243848

In [14]:
def avg_metric(task):
    aucs = [r[task][0] for r in fold_results]
    f1s  = [r[task][1] for r in fold_results]
    accs = [r[task][2] for r in fold_results]
    return np.mean(aucs), np.mean(f1s), np.mean(accs)

for task in ["late", "warn48", "submitted"]:
    auc, f1, acc = avg_metric(task)
    print(f"[AVG] {task}: AUC={auc:.4f}, F1={f1:.4f}, ACC={acc:.4f}")

[AVG] late: AUC=0.9663, F1=0.9217, ACC=0.9079
[AVG] warn48: AUC=0.9907, F1=0.9235, ACC=0.9550
[AVG] submitted: AUC=0.9699, F1=0.8398, ACC=0.9174


In [15]:
import pandas as pd
import numpy as np

# fold_results 리스트와 avg_metric 함수, 평균값 출력 부분이 이미 위 셀에 있다고 가정

# ==========================
# 1) Fold별 상세 표 만들기
# ==========================
rows = []
for fold_idx, res in enumerate(fold_results, start=1):
    for task in ["late", "warn48", "submitted"]:
        auc, f1, acc = res[task]
        rows.append({
            "fold": fold_idx,
            "task": task,
            "AUC": float(auc),
            "F1": float(f1),
            "ACC": float(acc),
        })

df_folds = pd.DataFrame(rows)

# ==========================
# 2) 평균값 표 만들기
# ==========================
mean_rows = []
for task in ["late", "warn48", "submitted"]:
    auc, f1, acc = avg_metric(task)
    mean_rows.append({
        "task": task,
        "AUC": auc,
        "F1": f1,
        "ACC": acc,
    })

df_mean = pd.DataFrame(mean_rows)

# ==========================
# 3) 출력
# ==========================
print("===== Fold별 성능 요약 =====")
display(df_folds.pivot(index="fold", columns="task", values="AUC"))
display(df_folds.pivot(index="fold", columns="task", values="F1"))
display(df_folds.pivot(index="fold", columns="task", values="ACC"))

print("\n===== 평균 성능 요약 =====")
display(df_mean)


===== Fold별 성능 요약 =====


task,late,submitted,warn48
fold,,,
1,0.964165,0.968499,0.990188
2,0.966210,0.970355,0.990715
3,0.966650,0.970157,0.990670
4,0.966453,0.969973,0.990699
5,0.967786,0.970757,0.991092


task,late,submitted,warn48
fold,,,
1,0.920218,0.834104,0.923059
2,0.921621,0.841375,0.923862
3,0.920381,0.840559,0.922293
4,0.921315,0.841011,0.923620
5,0.924814,0.842136,0.924710


task,late,submitted,warn48
fold,,,
1,0.905663,0.915894,0.954498
2,0.907530,0.917962,0.955064
3,0.906934,0.917472,0.954513
4,0.907845,0.916775,0.955244
5,0.911362,0.918843,0.955855



===== 평균 성능 요약 =====


,task,AUC,F1,ACC
0,late,0.966253,0.921670,0.907867
1,warn48,0.990673,0.923509,0.955035
2,submitted,0.969948,0.839837,0.917389


In [16]:
import pandas as pd

# === Baseline late 평균 성능 ===
baseline_late = {
    "AUC": 0.8237,
    "F1": 0.6435,
    "ACC": 0.8161
}

# === Multi-task MLP late 평균 성능 ===
mlp_late = {
    "AUC": 0.9663,
    "F1": 0.9217,
    "ACC": 0.9079
}

# === 비교 계산 ===
diff_late = {
    "AUC": mlp_late["AUC"] - baseline_late["AUC"],
    "F1": mlp_late["F1"] - baseline_late["F1"],
    "ACC": mlp_late["ACC"] - baseline_late["ACC"]
}

# === 출력용 표 만들기 ===
df_compare = pd.DataFrame({
    "Baseline": baseline_late,
    "MultiTask MLP": mlp_late,
    "Improvement": diff_late
})

print("===== Baseline 대비 Late 성능 개선 =====")
display(df_compare.style.format("{:.4f}"))


===== Baseline 대비 Late 성능 개선 =====


,Baseline,MultiTask MLP,Improvement
AUC,0.8237,0.9663,0.1426
F1,0.6435,0.9217,0.2782
ACC,0.8161,0.9079,0.0918
